In [ ]:
import spectHR as cs
import pyhrv

In [ ]:
cs.handler.show_logs()

In [ ]:
DataSet = cs.SpectHRDataset("Example Data/SUB_011.xdf", use_webdav=True)

In [ ]:
DataSet = cs.borderData(DataSet)
#DataSet.ecg = DataSet.ecg.slicetime(500, 570)

In [ ]:
DataSet = cs.filterECGData(DataSet, {"filterType": "highpass", "cutoff": .50})

In [ ]:
DataSet = cs.calcPeaks(DataSet)

In [ ]:
%matplotlib widget
GUI = cs.prepPlot2(DataSet, 500, 570)

In [ ]:
display(GUI)

In [ ]:
cs.poincare(DataSet, 0, 1200)

In [ ]:
DataSet.RTops.groupby('epoch')['ibi'].describe()